# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist,squareform
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
# from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from SampleLists.SampleFileFunctions import *
from TCR_feature_generation.publicSeqAnalysis import *

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'13052018'

# plan:

1. load TCR matrix from relevant dataset or prepare (minNshared=2)
2. filter for top sequences, remove outliers
3. calculate distance matrices - braycurtis or jaccard

4. load Mb data: with or without Nextera filtering
5. filter for top species+remove same person, remove outliers
6. 3. calculate distance matrices - braycurtis or jaccard

7. cross different TCR and Mb distance matrices, remove same person (only at this stage!) equalize sample names and order and calculate mantel test. 

# generate distance matrices for different TCR sharing matrices:

## load file and check for same-person pairs:

In [5]:
def prepare_TCRdfRA_TCRdfbinary_for_distMat(datasetFolder,datasetName,minNshared):
    #prepare folder to save resulting distance matrix:
    distMatFolder='%s/distanceMatrices' %datasetFolder
    if not isdir(distMatFolder):
        makedirs(distMatFolder)
    
    #load/generate TCRdf_RA file:
    TCRdf_RAFile='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA' %(datasetFolder,datasetName,
                                                                         minNshared)

    if isfile(TCRdf_RAFile):
        print 'loading TCRdf_RA...'
        TCRdf_RA=pd.read_pickle(TCRdf_RAFile)
    else:
        print 'preparing sharing Matrix'
        TCRdf_RA=gen_sharingMatrix_perDataset(datasetName,dataset_folder,minNsharedList,extractUniqueAA=False,getSharingStatistics=True)
    print 'done...'
    print TCRdf_RA.iloc[:5,:5]
    
    #load/generate TCRdf_binary file:
    TCRdfFile_binary='%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_binary' %(datasetFolder,datasetName,
                                                                         minNshared)
    if isfile(TCRdfFile_binary):
        print 'loading TCRdf_binary...'
        TCRdf_binary=pd.read_pickle(TCRdfFile_binary)
    else:
        print 'converting TCRdf_RA to binary...'
        TCRdf_binary = pd.DataFrame()
        print 'number of columns to convert is %s' %len(TCRdf_binary.columns.values)
        for n,column in enumerate(TCRdf_RA.columns.values):
            if n%1000==0:
                print n
            TCRdf_binary[column] = np.where(TCRdf_RA[column] > 0, 1, 0)
        TCRdf_binary = TCRdf_binary.set_index(TCRdf_RA.index)
        TCRdf_binary.to_pickle(TCRdfFile_binary)

    print 'done'
    print TCRdf_binary.iloc[:5,:5]
    
    #check same-person couples in dataset:
    print 'checking for same-person pairs within the dataset:'
    samePairs=[('BD617','BD838'),('BD705','BD714')]

    sampleList=TCRdf_RA.index
    print 'number of samples in the dataset - %s' %len(sampleList)

    for samplePair in samePairs:
        if (samplePair[0] in sampleList) and  (samplePair[1] in sampleList):
            print 'the sample pair %s and %s are in the dataset' %(samplePair[0],samplePair[1])
        elif samplePair[0] in sampleList:
            print 'only sample %s is in the dataset' %samplePair[0]
        elif samplePair[1] in sampleList:
            print 'only sample %s is in the dataset' %samplePair[1]
            
    return TCRdf_RA,TCRdf_binary

#### run for PNP515

In [21]:
#define parameters:
datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'
minNshared=2

#load files:
TCRdf_RA,TCRdf_binary=prepare_TCRdfRA_TCRdfbinary_for_distMat(datasetFolder,datasetName,minNshared)

loading TCRdf_RA...
done...
Sequence  *GPTHGYTF  *LYEQYF   CA  CA*NRGRNSPLHF  CA*TQYF
Sample                                                   
BD1             0.0      0.0  0.0            0.0      0.0
BD10            0.0      0.0  0.0            0.0      0.0
BD101           0.0      0.0  0.0            0.0      0.0
BD105           0.0      0.0  0.0            0.0      0.0
BD106           0.0      0.0  0.0            0.0      0.0
loading TCRdf_binary...
done
        *GPTHGYTF  *LYEQYF  CA  CA*NRGRNSPLHF  CA*TQYF
Sample                                                
BD1             0        0   0              0        0
BD10            0        0   0              0        0
BD101           0        0   0              0        0
BD105           0        0   0              0        0
BD106           0        0   0              0        0
checking for same-person pairs within the dataset:
number of samples in the dataset - 392
the sample pair BD617 and BD838 are in the dataset
only sample

## filter for top sequences,  remove outliers, and generate distMats:

In [22]:
def gen_distMat_for_TCRdfs(TCRdf_RA,TCRdf_binary,topNseqs, outlierSTD,distMetricRA, distMetricBinary,datasetFolder):
    
    #prepare folder to save resulting distance matrix:
    distMatFolder='%s/distanceMatrices' %datasetFolder
    if not isdir(distMatFolder):
        makedirs(distMatFolder)
    
    if topNseqs is not None:
        # #filter SEQ binary file to include only sequences shared by more than NsharedSamplesForSeqs:
        print topNseqs
        print 'filtering sequence df to include only top %s sequences ' % topNseqs
        print topNseqs
        seqCount = TCRdf_binary.sum().sort_values(ascending=False)
        print topNseqs
        topSeqs = list(seqCount.index[:topNseqs])
        print topNseqs
    #     print topSeqs
        print len(topSeqs)
        TCRdf_RA = TCRdf_RA.loc[:, topSeqs]
        TCRdf_binary = TCRdf_binary.loc[:, topSeqs]
    else:   
        print 'TCRdfs were not filtered for top sequences...'
#     print 'TCRdf binary shape is:'
#     print TCRdf_binary.shape

#     print 'TCRdf RA shape is:'
#     print TCRdf_RA.shape
        
    ### remove outiers from TCRdf_RA
    if outlierSTD is not None:  # replace outliers with trimming value
        print 'filtering outliers'
        TCRdf_RA = filter_outliers(df=TCRdf_RA, outlierSTD=outlierSTD, columnList=None,trim=True)
    print 'TCRdf binary shape is:'
    print TCRdf_binary.shape

    print 'TCRdf RA shape is:'
    print TCRdf_RA.shape
    
    ### Generate distance matrices - RA data:
    #generate distance series:
    print 'generating condensed distance matrix for RA data - %s' %distMetricRA
    TCRdf_RA_distMat_condensed=pd.Series(pdist(TCRdf_RA, distMetricRA))

    # add sample names:
    sample1List=[]
    sample2List=[]

    count=0
    for i in range(len(TCRdf_RA.index)):
        for j in range(i+1,len(TCRdf_RA.index)):
            sample1=TCRdf_RA.index[i]
            sample2=TCRdf_RA.index[j]
            sample1List.append(sample1)
            sample2List.append(sample2)
            count=count+1
#     print count
    TCRdf_RA_distMat_condensed_org=pd.DataFrame({'sample1':sample1List,'sample2':sample2List,'dist':TCRdf_RA_distMat_condensed})
    print 'top similar pairs for TCRdf with topNseqs=%s' %topNseqs
    print TCRdf_RA_distMat_condensed_org.sort_values(by='dist').head(10)
    TCRdf_RA_distMat_file_condensed='%s/distMat_RA_%s_topNseqs%s_OLstd%s_condensed' %(distMatFolder,distMetricRA,topNseqs,outlierSTD)
    TCRdf_RA_distMat_condensed_org.to_pickle(TCRdf_RA_distMat_file_condensed)
    
    #GENERATE SQUAREFORM DISTANCE MATRIX AND SAVE:
    print 'generating squareform distance matrix for RA data - %s' %distMetricRA
    TCRdf_RA_distMat_square = pd.DataFrame(squareform(pdist(TCRdf_RA, distMetricRA)),columns=TCRdf_RA.index,index=TCRdf_RA.index)
#     print TCRdf_RA_distMat_square.iloc[:5,:5]

    TCRdf_RA_distMat_file='%s/distMat_RA_%s_topNseqs%s_OLstd%s' %(distMatFolder,distMetricRA,topNseqs,outlierSTD)
    TCRdf_RA_distMat_square.to_pickle(TCRdf_RA_distMat_file)
    
    ### Generate distance matrices - binary data :
    #generate distance series:
    print 'generating condensed distance matrix for binary data - %s' %distMetricBinary
    TCRdf_binary_distMat_condensed=pd.Series(pdist(TCRdf_binary,distMetricBinary))

    # add sample names:
    sample1List=[]
    sample2List=[]

    count=0
    for i in range(len(TCRdf_binary.index)):
        for j in range(i+1,len(TCRdf_binary.index)):
            sample1=TCRdf_binary.index[i]
            sample2=TCRdf_binary.index[j]
            sample1List.append(sample1)
            sample2List.append(sample2)
            count=count+1
    print count
    TCRdf_binary_distMat_condensed_org=pd.DataFrame({'sample1':sample1List,'sample2':sample2List,'dist':TCRdf_binary_distMat_condensed})
    print 'top similar pairs for TCRdf binary with topNseqs=%s' %topNseqs
    print TCRdf_binary_distMat_condensed_org.sort_values(by='dist').head(10)
    TCRdf_binary_distMat_file_condensed='%s/distMat_binary_%s_topNseqs%s_condensed' %(distMatFolder,distMetricBinary,topNseqs)
    TCRdf_RA_distMat_condensed_org.to_pickle(TCRdf_binary_distMat_file_condensed)
    
    #GENERATE SQUAREFORM DISTANCE MATRIX AND SAVE:
    print 'generating squareform distance matrix for binary data - %s' %distMetricBinary
    TCRdf_binary_distMat_square = pd.DataFrame(squareform(pdist(TCRdf_binary, distMetricBinary)),columns=TCRdf_binary.index,index=TCRdf_binary.index)
#     print TCRdf_binary_distMat_square.iloc[:5,:5]

    TCRdf_binary_distMat_file='%s/distMat_binary_%s_topNseqs%s' %(distMatFolder,distMetricBinary,topNseqs)
    TCRdf_binary_distMat_square.to_pickle(TCRdf_binary_distMat_file)
    
    return TCRdf_RA_distMat_square,TCRdf_binary_distMat_square
    
    

#### run for different topNseqs, outlierSTD and metrics;

In [23]:
#braycurtis+jaccard:
datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
topNseqsList=[None,100000,10000,1000,20]
outlierSTDlist=[None]

TCRdf_RA=TCRdf_RA
TCRdf_binary=TCRdf_binary

distMetricRA='braycurtis'
distMetricBinary='jaccard'

for topNseqs in topNseqsList:
    for outlierSTD in outlierSTDlist:
        print  'topNseqs=%s,outlierSTD=%s' %(topNseqs,outlierSTD)
        TCRdf_RA_distMat_square,TCRdf_binary_distMat_square=\
gen_distMat_for_TCRdfs(TCRdf_RA,TCRdf_binary,topNseqs, outlierSTD,distMetricRA, distMetricBinary,datasetFolder)
    

topNseqs=None,outlierSTD=None
TCRdfs were not filtered for top sequences...
TCRdf binary shape is:
(392, 276117)
TCRdf RA shape is:
(392, 276117)
generating condensed distance matrix for RA data - braycurtis
top similar pairs for TCRdf with topNseqs=None
           dist sample1 sample2
62178  0.568032   BD617   BD838
45017  0.641662   BD431   BD440
68657  0.727990   BD788   BD809
65909  0.733858   BD699   BD706
67424  0.769906   BD714   BD913
24828  0.851414   BD245   BD788
29050  0.886873   BD337   BD339
62471  0.918935   BD620   BD714
54069  0.919974   BD505   BD547
32207  0.924775   BD355   BD603
generating squareform distance matrix for RA data - braycurtis
generating condensed distance matrix for binary data - jaccard
76636
top similar pairs for TCRdf binary with topNseqs=None
           dist sample1 sample2
45017  0.639786   BD431   BD440
67424  0.784925   BD714   BD913
68657  0.802650   BD788   BD809
65909  0.808099   BD699   BD706
62178  0.811899   BD617   BD838
29050  0.922936

# generate distance matrices for different Mb sharing matrices:

## load file:

In [9]:
def prepare_MbDFRA_MbDFbinary_for_distMat(libPrepMethod):
    
    #load/generate MbDF_RA file:
    MbFolder='%s/MicrobiomeDataTables/FilteredAndMergedOnBD' %MyPath
    MbDF_RAFile='%s/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekTrue_filterMinimalRead9000000_\
libMeth%s_meannSTDNonenMinSamplesNone_fOutlierSampleFalse_fSamePersonFalse' %(MbFolder,libPrepMethod)
    
    if isfile(MbDF_RAFile):
        print 'loading MbDF_RA...'
        MbDF_RA=pd.read_pickle(MbDF_RAFile)
        print MbDF_RA.iloc[:5,:5]
    else:
        print 'MbDF file doesnt exist'
        
 ##################add option to prepare file?################################       
        
    print 'done...'
   
    
    
   
    #load/generate MbDF_binary file:
    MbDFFile_binary='%s_RA' %MbDF_RAFile
    
    if isfile(MbDFFile_binary):
        print 'loading MbDF_binary...'
        MbDF_binary=pd.read_pickle(MbDFFile_binary)
    else:
        print 'converting MbDF_RA to binary...'
        MbDF_binary = pd.DataFrame()
        print 'number of columns to convert is %s' %len(MbDF_binary.columns.values)
        for n,column in enumerate(MbDF_RA.columns.values):
            if n%1000==0:
                print n
            MbDF_binary[column] = np.where(MbDF_RA[column] > 0, 1, 0)
        MbDF_binary = MbDF_binary.set_index(MbDF_RA.index)
        MbDF_binary.to_pickle(MbDFFile_binary)

    print 'done'
    print MbDF_binary.iloc[:5,:5]
    
    #filter to include only species that appear in more than one sample:
    print 'filtering species df according to minimal number of shared samples by species=2'
    speciesCount = MbDF_binary.sum()
    selectedSpecies = list(speciesCount[speciesCount > 1].index)
    print len(selectedSpecies)
    MbDF_RA = MbDF_RA[selectedSpecies]
    MbDF_binary=MbDF_binary[selectedSpecies]
    
    
    #check same-person couples in dataset:
   #check same-person couples in dataset:
    print 'checking for same-person pairs within the dataset:'
    samePairs=[('BD617','BD838'),('BD705','BD714')]

    sampleList=MbDF_RA.index
    print 'number of samples in the dataset - %s' %len(sampleList)

    for samplePair in samePairs:
        if (samplePair[0] in sampleList) and  (samplePair[1] in sampleList):
            print 'the sample pair %s and %s are in the dataset' %(samplePair[0],samplePair[1])
        elif samplePair[0] in sampleList:
            print 'only sample %s is in the dataset' %samplePair[0]
        elif samplePair[1] in sampleList:
            print 'only sample %s is in the dataset' %samplePair[1]
            
    return MbDF_RA,MbDF_binary

In [14]:
libPrepMethod='Nextera'

MbDF_RA_libNone,MbDF_binary_libNone=prepare_MbDFRA_MbDFbinary_for_distMat(libPrepMethod)

loading MbDF_RA...
       Abiotrophia_defectiva  Acidaminococcus_fermentans  \
BD                                                         
BD1                      0.0                         0.0   
BD10                     0.0                         0.0   
BD105                    0.0                         0.0   
BD106                    0.0                         0.0   
BD107                    0.0                         0.0   

       Acidaminococcus_intestini  Acidaminococcus_sp_BV3L6  \
BD                                                           
BD1                          0.0                       0.0   
BD10                         0.0                       0.0   
BD105                        0.0                       0.0   
BD106                        0.0                       0.0   
BD107                        0.0                       0.0   

       Acidaminococcus_sp_D21  
BD                             
BD1                       0.0  
BD10                      0.0

In [15]:
MbDF_RA_libNone.shape

(223, 345)

## filter for top sequences,  remove outliers, and generate distMats:

In [16]:
def gen_distMat_for_MbDF(MbDF_RA,MbDF_binary,topNspecies, outlierSTD,distMetricRA, distMetricBinary,libPrepMethod):
    
    #prepare folder to save resulting distance matrix:
    distMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
    if not isdir(distMatFolder):
        makedirs(distMatFolder)
    
    if topNspecies is not None:
        # #filter MbDF file to include only sequences shared by more than NsharedSamplesForSeqs:
        print topNspecies
        print 'filtering sequence df to include only top %s species ' % topNspecies
        speciesCount = MbDF_binary.sum().sort_values(ascending=False)
        topSpecies = list(speciesCount.index[:topNspecies])
        print len(topSpecies)
        MbDF_RA = MbDF_RA.loc[:, topSpecies]
        MbDF_binary = MbDF_binary.loc[:, topSpecies]
    else:   
        print 'MbDFs were not filtered for top species...'
        
    ### remove outiers from MbDF_RA
    if outlierSTD is not None:  # replace outliers with trimming value
        print 'filtering outliers'
        MbDF_RA = filter_outliers(df=MbDF_RA, outlierSTD=outlierSTD, columnList=None,trim=True)
    print 'MbDF binary shape is:'
    print MbDF_binary.shape

    print 'MbDF RA shape is:'
    print MbDF_RA.shape
    
    ### Generate distance matrices - RA data:
    #generate distance series:
    print 'generating condensed distance matrix for RA data - %s' %distMetricRA
    MbDF_RA_distMat_condensed=pd.Series(pdist(MbDF_RA, distMetricRA))

    # add sample names:
    sample1List=[]
    sample2List=[]

    count=0
    for i in range(len(MbDF_RA.index)):
        for j in range(i+1,len(MbDF_RA.index)):
            sample1=MbDF_RA.index[i]
            sample2=MbDF_RA.index[j]
            sample1List.append(sample1)
            sample2List.append(sample2)
            count=count+1
#     print count
    MbDF_RA_distMat_condensed_org=pd.DataFrame({'sample1':sample1List,'sample2':sample2List,'dist':MbDF_RA_distMat_condensed})
    print 'top similar pairs for MbDF_RA with topNspecies=%s' %topNspecies
    print MbDF_RA_distMat_condensed_org.sort_values(by='dist').head(10)
    MbDF_RA_distMat_file_condensed='%s/distMat_MbDF_RA_%s_topNspecies%s_OLstd%s_libPrep%s_condensed' %(distMatFolder,
                                                        distMetricRA,topNspecies,outlierSTD,libPrepMethod)
    MbDF_RA_distMat_condensed_org.to_pickle(MbDF_RA_distMat_file_condensed)
    
    #GENERATE SQUAREFORM DISTANCE MATRIX AND SAVE:
    print 'generating squareform distance matrix for RA data - %s' %distMetricRA
    MbDF_RA_distMat_square = pd.DataFrame(squareform(pdist(MbDF_RA, distMetricRA)),columns=MbDF_RA.index,index=MbDF_RA.index)
#     print MbDF_RA_distMat_square.iloc[:5,:5]

    MbDF_RA_distMat_file='%s/distMat_MbDF_RA_%s_topNspecies%s_OLstd%s_libPrep%s' %(distMatFolder,
                                                        distMetricRA,topNspecies,outlierSTD,libPrepMethod)
    MbDF_RA_distMat_square.to_pickle(MbDF_RA_distMat_file)
    
    ### Generate distance matrices - binary data :
    #generate distance series:
    print 'generating condensed distance matrix for binary data - %s' %distMetricBinary
    MbDF_binary_distMat_condensed=pd.Series(pdist(MbDF_binary,distMetricBinary))

    # add sample names:
    sample1List=[]
    sample2List=[]

    count=0
    for i in range(len(MbDF_binary.index)):
        for j in range(i+1,len(MbDF_binary.index)):
            sample1=MbDF_binary.index[i]
            sample2=MbDF_binary.index[j]
            sample1List.append(sample1)
            sample2List.append(sample2)
            count=count+1
    print count
    MbDF_binary_distMat_condensed_org=pd.DataFrame({'sample1':sample1List,'sample2':sample2List,'dist':MbDF_binary_distMat_condensed})
    print MbDF_binary_distMat_condensed_org.sort_values(by='dist').head(10)
    MbDF_binary_distMat_file_condensed='%s/distMat_MbDF_binary_%s_topNspecies%s_libPrep%s_condensed' %(distMatFolder,
                                                        distMetricBinary,topNspecies,libPrepMethod)
    MbDF_binary_distMat_condensed_org.to_pickle(MbDF_binary_distMat_file_condensed)
    
    #GENERATE SQUAREFORM DISTANCE MATRIX AND SAVE:
    print 'generating squareform distance matrix for binary data - %s' %distMetricBinary
    MbDF_binary_distMat_square = pd.DataFrame(squareform(pdist(MbDF_binary, distMetricBinary)),columns=MbDF_binary.index,index=MbDF_binary.index)


    MbDF_binary_distMat_file='%s/distMat_MbDF_binary_%s_topNspecies%s_libPrep%s' %(distMatFolder,
                                                                distMetricBinary,topNspecies,libPrepMethod)
    MbDF_binary_distMat_square.to_pickle(MbDF_binary_distMat_file)
    
    return MbDF_RA_distMat_square,MbDF_binary_distMat_square
    
    

#### run for different topNseqs, outlierSTD and metrics;

In [17]:
#braycurtis+jaccard:
topNspeciesList=[None,250,100,20]
outlierSTDlist=[None]

MbDF_RA=MbDF_RA_libNone
MbDF_binary=MbDF_binary_libNone

distMetricRA='braycurtis'
distMetricBinary='jaccard'

for topNspecies in topNspeciesList:
    for outlierSTD in outlierSTDlist:
        print  'topNspecies=%s,outlierSTD=%s' %(topNspecies,outlierSTD)
        MbDF_RA_distMat_square,MbDF_binary_distMat_square=\
gen_distMat_for_MbDF(MbDF_RA,MbDF_binary,topNspecies, outlierSTD,distMetricRA, distMetricBinary,libPrepMethod)
    

topNspecies=None,outlierSTD=None
MbDFs were not filtered for top species...
MbDF binary shape is:
(223, 345)
MbDF RA shape is:
(223, 345)
generating condensed distance matrix for RA data - braycurtis
top similar pairs for MbDF_RA with topNspecies=None
           dist sample1 sample2
18994  0.000013   BD476   BD504
19617  0.000024   BD483   BD506
7847   0.000028    BD35   BD695
17976  0.000038   BD454   BD477
9254   0.000038   BD371   BD488
17198  0.000040   BD444   BD725
2053   0.000041   BD113   BD446
1079   0.000044   BD107     BD9
11191  0.000044   BD385   BD888
5140   0.000045    BD24   BD469
generating squareform distance matrix for RA data - braycurtis
generating condensed distance matrix for binary data - jaccard
24753
           dist sample1 sample2
3013   0.266055    BD13   BD923
24449  0.268519   BD895   BD923
17863  0.272727   BD453   BD480
19435  0.274510   BD480    BD70
24739  0.280374   BD921   BD923
2171   0.288288   BD113   BD923
17963  0.289474   BD453   BD923
24343  0

# run mantel tests

first: TCR15000-top10000 X microbiome no nextera filtering

then:
TCR150000-top1000,100000,none, TCR9000-1000,10000,10000, full-10000,100000
X
microbiome with/without nextera filtering
X
binary,RA

## run for RA data: dataset15000, different topNspecies, topNsequences,libPrepMethod

In [30]:
#variables to edit:
datasetFolder='%s/TCR_real_data' %MyPath
datasetName='PNP515'
topNseqsList=[None,100000,10000,1000,20]
topNspeciesList=[None,250,100,20]
distMetricRA='braycurtis'
outlierSTD=None
libPrepMethodList=[None,'Nextera']

MantelFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/TCR_Mb_Mantel_dfs_updated' %MyPath
if not isdir(MantelFolder):
    makedirs(MantelFolder)
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True



#derived variables:
count=0
for topNseqs in topNseqsList:
    for topNspecies in topNspeciesList:
        for libPrepMethod in libPrepMethodList:
            
            MantelName='MantelDF_%s_%seq_%species_libMethod%s_%s_olSTD%s_reSameUser%s' %(datasetName,topNseqs,topNspecies,
                                                    libPrepMethod, distMetricRA,outlierSTD,removeSameUser)                                                                    
            MantelFile='%s/%s' %(MantelFolder,MantelName)
            existingMantels=[f for f in listdir(MantelFolder) if isfile(join(MantelFolder, f))]

            if MantelName not in existingMantels:

            
                print count, topNseqs,topNspecies,libPrepMethod
                TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
                MbdistMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
                distMat1name='distMat_RA_%s_topNseqs%s_OLstd%s' %(distMetricRA,topNseqs,outlierSTD)
                distMat2name='distMat_MbDF_RA_%s_topNspecies%s_OLstd%s_libPrep%s' %(distMetricRA,topNspecies,outlierSTD,libPrepMethod)


                distMat1file='%s/%s' %(TCRdistMatFolder,distMat1name)
                distMat2file='%s/%s' %(MbdistMatFolder,distMat2name)
                distMat1=pd.read_pickle(distMat1file)
                distMat2=pd.read_pickle(distMat2file)


                mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
                mantelDF.loc[0,'datasetName']=datasetName
                mantelDF.loc[0,'topNseqs']=topNseqs
                mantelDF.loc[0,'topNspecies']=topNspecies
                mantelDF.loc[0,'distMetricRA']=distMetricRA
                mantelDF.loc[0,'outlierSTD']=outlierSTD
                mantelDF.loc[0,'libPrepMethod']=libPrepMethod
                mantelDF.loc[0,'removeSameUser']=removeSameUser


                mantelDF.to_pickle(MantelFile)
            else:
                print 'mantel already exist'
            
            count=count+1

mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
32 20 None None
original x array shape is 515_515
original y array shape is 447_447
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have mor

DistanceMatrixError: Data must be symmetric.

## run for binary: dataset15000, different topNspecies, topNsequences,libPrepMethod

In [32]:
#variables to edit:
datasetFolder='%s/TCR_real_data' %MyPath
datasetName='PNP515'
topNseqsList=[None,100000,10000,1000,20]
topNspeciesList=[None,250,100,20]
distMetricRA='jaccard'
outlierSTD=3
libPrepMethodList=[None,'Nextera']

MantelFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/TCR_Mb_Mantel_dfs_updated' %MyPath
if not isdir(MantelFolder):
    makedirs(MantelFolder)
method='spearman'
n_permut=999
alternative='greater'
removeSameUser=True



#derived variables:
count=0
for topNseqs in topNseqsList:
    for topNspecies in topNspeciesList:
        for libPrepMethod in libPrepMethodList:
            
            MantelName='MantelDF_%s_%seq_%species_libMethod%s_%s_olSTD%s_reSameUser%s' %(datasetName,topNseqs,topNspecies,
                                                    libPrepMethod, distMetricRA,outlierSTD,removeSameUser)                                                                    
            MantelFile='%s/%s' %(MantelFolder,MantelName)
            existingMantels=[f for f in listdir(MantelFolder) if isfile(join(MantelFolder, f))]

            if MantelName not in existingMantels:

            
                print count, topNseqs,topNspecies,libPrepMethod
                TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
                MbdistMatFolder='%s/MicrobiomeDataTables/distanceMatrices' %MyPath
                distMat1name='distMat_binary_%s_topNseqs%s' %(distMetricRA,topNseqs)
                distMat2name='distMat_MbDF_binary_%s_topNspecies%s_libPrep%s' %(distMetricRA,topNspecies,libPrepMethod)


                distMat1file='%s/%s' %(TCRdistMatFolder,distMat1name)
                distMat2file='%s/%s' %(MbdistMatFolder,distMat2name)
                distMat1=pd.read_pickle(distMat1file)
                distMat2=pd.read_pickle(distMat2file)


                mantelDF=mantelTest_feature_phenotype(distMat1, distMat2,distMat1name, distMat2name, MantelFolder,method,n_permut,alternative,removeSameUser)
                mantelDF.loc[0,'datasetName']=datasetName
                mantelDF.loc[0,'topNseqs']=topNseqs
                mantelDF.loc[0,'topNspecies']=topNspecies
                mantelDF.loc[0,'distMetricRA']=distMetricRA
                mantelDF.loc[0,'outlierSTD']=outlierSTD
                mantelDF.loc[0,'libPrepMethod']=libPrepMethod
                mantelDF.loc[0,'removeSameUser']=removeSameUser


                mantelDF.to_pickle(MantelFile)
            else:
                print 'mantel already exist'
            
            count=count+1

mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
mantel already exist
8 100000 None None
original x array shape is 515_515
original y array shape is 447_447
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
new x array shape is 

-0.023049678879931073 0.748
saving mantel results
DONE!
13 100000 100 Nextera
original x array shape is 515_515
original y array shape is 223_223
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 223
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 223_223
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 223
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 223_223
new x array shape is 223_223
new y array shape is 223_223
            BD1      BD10     BD105     BD106     BD107
BD                                    

0.05040838674344798 0.155
saving mantel results
DONE!
18 10000 250 None
original x array shape is 515_515
original y array shape is 447_447
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
new x array shape is 445_445
new y array shape is 445_445
            BD1      BD10     BD101     BD105     BD106
BD                    

       BD1  BD10  BD101  BD105  BD106
BD                                   
BD1    0.0   0.0    0.0    0.0    0.0
BD10   0.0   0.0    0.0    0.0    0.0
BD101  0.0   0.0    0.0    0.0    0.0
BD105  0.0   0.0    0.0    0.0    0.0
BD106  0.0   0.0    0.0    0.0    0.0
make sure matrices dimensions and heads are identical!
running mantel test...
-0.0282990231866125 0.758
saving mantel results
DONE!
23 10000 20 Nextera
original x array shape is 515_515
original y array shape is 223_223
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
0 UserIDs have more than one BD
groupedDF length before dropping same person samples is 223
removing UserIDs: []
removing BD: []
groupedDF shape after dropping same person samples is 223_223
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
0 U

0.042891078991550126 0.158
saving mantel results
DONE!
28 1000 100 None
original x array shape is 515_515
original y array shape is 447_447
loading and processing distance matrix files...
verifying same samples in compared distance matrices:
filtering same person samples
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
loading BD_FD table
number of BD-UserID pairs is 1100
merging UserID info to BDgrouped table:
1 UserIDs have more than one BD
groupedDF length before dropping same person samples is 446
removing UserIDs: [447]
removing BD: [u'BD838']
groupedDF shape after dropping same person samples is 445_445
new x array shape is 445_445
new y array shape is 445_445
            BD1      BD10     BD101     BD105     BD106
BD                    

DistanceMatrixError: Data must be symmetric.

*** conclusions: ***
no significant association between TCR and microbiome distance matrices! for all matrices combination tested

# merge results:

In [46]:
MantelFolder='%s/TCR_real_data/TCR_mb_results/DATA_SUMMARY_ALL_DATASETS/TCR_Mb_Mantel_dfs_updated' %MyPath

In [47]:
allmantel=concat_summarizing_dfs(MantelFolder)

number of dfs in directory: 176
the length of df list is 176


In [48]:
allmantel.sort_values(by='p_value')

,distMat1name,distMat2name,method,n_permut,alternative,corr_coeff,p_value,N,datasetName,topNseqs,topNspecies,distMetricRA,outlierSTD,libPrepMethod,removeSameUser
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.058703,0.117,223.0,PNP515,NaN,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.053132,0.124,223.0,PNP515,NaN,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.058703,0.126,223.0,PNP515,100000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs10000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.056475,0.127,223.0,PNP515,10000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.050792,0.135,223.0,PNP515,NaN,250.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.050792,0.146,223.0,PNP515,100000.0,250.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs1000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.050796,0.147,223.0,PNP515,1000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.053132,0.153,223.0,PNP515,100000.0,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs10000,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.050408,0.155,223.0,PNP515,10000.0,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs1000,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.042891,0.158,223.0,PNP515,1000.0,250.0,jaccard,3.0,Nextera,True


In [50]:
allmantel.sort_values(by='corr_coeff',ascending=False)

,distMat1name,distMat2name,method,n_permut,alternative,corr_coeff,p_value,N,datasetName,topNseqs,topNspecies,distMetricRA,outlierSTD,libPrepMethod,removeSameUser
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.058703,0.126,223.0,PNP515,100000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.058703,0.117,223.0,PNP515,NaN,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs10000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.056475,0.127,223.0,PNP515,10000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.053132,0.153,223.0,PNP515,100000.0,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.053132,0.124,223.0,PNP515,NaN,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs1000,distMat_MbDF_binary_jaccard_topNspecies100_lib...,spearman,999.0,greater,0.050796,0.147,223.0,PNP515,1000.0,100.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs100000,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.050792,0.146,223.0,PNP515,100000.0,250.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqsNone,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.050792,0.135,223.0,PNP515,NaN,250.0,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs10000,distMat_MbDF_binary_jaccard_topNspeciesNone_li...,spearman,999.0,greater,0.050408,0.155,223.0,PNP515,10000.0,NaN,jaccard,3.0,Nextera,True
0,distMat_binary_jaccard_topNseqs10000,distMat_MbDF_binary_jaccard_topNspecies250_lib...,spearman,999.0,greater,0.048124,0.171,223.0,PNP515,10000.0,250.0,jaccard,3.0,Nextera,True


# fisher and correlation test for top seq by top species binary data
according to Eran's suggestion - 130518:

I ran fisher and spearman tests with 10000 permutations on 130518

In [63]:
# real correlation test results (no permutations):
file1='%s/TCR_real_data/SubSampled15000data_rep2/TCR_mb_results/permCorrTest_NoneNoneNoneNone_stdNone/real_result_df' %MyPath
spearmanResults=pd.read_pickle(file1)

In [66]:
spearmanResults.head()

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies
0,Acidaminococcus_fermentans,CASRGQGNTEAFF,0.100038,38.0,11.0,3.0
0,Acidaminococcus_fermentans,CASRQAGETQYF,0.076965,46.0,11.0,3.0
0,Acidaminococcus_fermentans,CASSAQETQYF,-0.061779,43.0,11.0,0.0
0,Acidaminococcus_fermentans,CASSASTDTQYF,-0.003801,37.0,11.0,1.0
0,Acidaminococcus_fermentans,CASSASYEQYF,0.051218,59.0,11.0,3.0


In [71]:
spearmanResults.sort_values(by='r_real',ascending=False).head(20)

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies
0,Slackia_piriformis,CASSLGDEQYF,0.322816,42.0,12.0,8.0
0,Oscillibacter_sp_KLE_1745,CASSLSLNTEAFF,0.283280,47.0,8.0,6.0
0,Butyricicoccus_pullicaecorum,CASSLTSGGYNEQFF,0.267577,38.0,3.0,3.0
0,Bacteroides_sp_1_1_30,CASSFLETQYF,0.267222,38.0,5.0,4.0
0,Collinsella_tanakaei,CASSLGQGNEKLFF,0.265845,59.0,7.0,6.0
0,Porphyromonas_bennonis,CASSRTVNTEAFF,0.263758,40.0,13.0,7.0
0,Salivirus_A,CASSLSGTDTQYF,0.263301,40.0,3.0,3.0
0,Acidaminococcus_sp_HPA0509,CASSSRTDTQYF,0.260908,42.0,3.0,3.0
0,Cetobacterium_somerae,CASSLGRGYEQYF,0.255684,39.0,3.0,3.0
0,Pseudoflavonifractor_capillosus,CASSLGANTGELFF,0.253344,56.0,64.0,23.0


In [ ]:
#define dataset:
datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2

In [58]:
#load TCRdf data and cut for only samples appearing in more than 10% of samples:
minNshared=2

#load files:
TCRdf_RA,TCRdf_binary=prepare_TCRdfRA_TCRdfbinary_for_distMat(datasetFolder,datasetName,minNshared)

NsharedSamplesForSeqs=round(len(TCRdf_binary)*0.1,0)
print '10 percent of samples is %s' %NsharedSamplesForSeqs

seqCount = TCRdf_binary.sum()
selectedSeqs = list(seqCount[seqCount > NsharedSamplesForSeqs].index)  
TCRdf_binary = TCRdf_binary[selectedSeqs]

print TCRdf_binary.shape


loading TCRdf_RA...
done...
Sequence  *GPTHGYTF  *LYEQYF   CA  CA*NRGRNSPLHF  CA*TQYF
Sample                                                   
BD1             0.0      0.0  0.0            0.0      0.0
BD10            0.0      0.0  0.0            0.0      0.0
BD101           0.0      0.0  0.0            0.0      0.0
BD105           0.0      0.0  0.0            0.0      0.0
BD106           0.0      0.0  0.0            0.0      0.0
loading TCRdf_binary...
done
        *GPTHGYTF  *LYEQYF  CA  CA*NRGRNSPLHF  CA*TQYF
Sample                                                
BD1             0        0   0              0        0
BD10            0        0   0              0        0
BD101           0        0   0              0        0
BD105           0        0   0              0        0
BD106           0        0   0              0        0
checking for same-person pairs within the dataset:
number of samples in the dataset - 392
the sample pair BD617 and BD838 are in the dataset
only sample

In [60]:
#load MbDF and cut for only species with RA>0.0001
libPrepMethod=None
MbDF_RA,MbDF_binary=prepare_MbDFRA_MbDFbinary_for_distMat(libPrepMethod)


print 'converting MbDF_RA to binary...'
MbDF_binary_cutoff = pd.DataFrame()
print 'number of columns to convert is %s' %len(MbDF_RA.columns.values)
for n,column in enumerate(MbDF_RA.columns.values):
    if n%200==0:
        print n
    MbDF_binary_cutoff[column] = np.where(MbDF_RA[column] > 0.0001, 1, 0)
    if MbDF_binary_cutoff[column].sum()==0:
        print 'species %s is being dropped...' %column
        MbDF_binary_cutoff=MbDF_binary_cutoff.drop(column,axis=1)
MbDF_binary_cutoff = MbDF_binary_cutoff.set_index(MbDF_RA.index)

print MbDF_binary_cutoff.shape


loading MbDF_RA...
       Abiotrophia_defectiva  Acidaminococcus_fermentans  \
BD                                                         
BD1                      0.0                         0.0   
BD10                     0.0                         0.0   
BD101                    0.0                         0.0   
BD105                    0.0                         0.0   
BD106                    0.0                         0.0   

       Acidaminococcus_intestini  Acidaminococcus_sp_BV3L6  \
BD                                                           
BD1                          0.0                       0.0   
BD10                         0.0                       0.0   
BD101                        0.0                       0.0   
BD105                        0.0                       0.0   
BD106                        0.0                       0.0   

       Acidaminococcus_sp_D21  
BD                             
BD1                       0.0  
BD10                      0.0